# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [78]:
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os
import sys
from pathlib import Path
from torchinfo import summary

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [41]:
from CheXpert.race_prediction.dataset import CheXpertRaceDataset
from CheXpert.disease_prediction.dataset import CheXpertDiseaseDataset
from shared_utils import vprint, to_gpu, add_mean_to_list, Mode
import shared_utils
from CheXpert.disease_prediction.utils import Configs as disease_configs
from CheXpert.race_prediction.utils import Configs as race_configs
from MIMIC_CXR.dataset import CXRDataset
from MIMIC_CXR.utils import Configs as cxr_configs

# Configs

In [59]:
@dataclass
class Configs:
    CXR_DATA_DIR = os.path.join("data", "MIMIC-CXR-JPG")
    CXR_VALID_LABELS_FILENAME = "valid.csv"
    CXR_FILENAMES = cxr_configs.CXR_FILENAMES
    CHEXPERT_DATA_DIR = os.path.join("data", "CheXpert", "CheXpert-v1.0-small")
    CHEXPERT_DISEASE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "disease_prediction", "trained_models")
    CHEXPERT_RACE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "race_prediction", "trained_models")
    CHEXPERT_VALID_LABELS_FILENAME = "valid.csv"
    CHEXPERT_DEMO_FILENAME = "CHEXPERT DEMO.csv"
    DISEASE_ANNOTATIONS_COLUMNS = disease_configs.ANNOTATIONS_COLUMNS
    RACE_ANNOTATIONS_COLUMNS = race_configs.ANNOTATIONS_COLUMNS
    NUM_DISEASE_CLASSES = disease_configs.NUM_CLASSES
    NUM_RACE_CLASSES = race_configs.NUM_CLASSES
    CHEXPERT_RACE_DICT = race_configs.RACE_DICT
    BATCH_SIZE = 4
    SEED = 123
    VERBOSE = 1
    VALID_SIZE_DEBUG = -1

In [7]:
shared_utils.set_seed(Configs.SEED)

In [8]:
if torch.cuda.is_available():
    vprint(f"Memory info: {torch.cuda.mem_get_info()[0]/10e8:.1f} GB free GPU.", Configs)
else: 
    vprint(f"No GPU Memory.", Configs)

2022-07-23 19:57: Memory info: 8.5 GB free GPU.


In [9]:
valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Disease Prediction 

## Validaiton Dataloaders

In [14]:
cxp_disease_valid_dataset = CheXpertDiseaseDataset(data_dir=Configs.CHEXPERT_DATA_DIR, 
                                                   labels_filename=Configs.CHEXPERT_VALID_LABELS_FILENAME,
                                                   transform=valid_transform)
cxp_disease_valid_dataset.df_labels = cxp_disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxp_disease_valid_dataloader = DataLoader(cxp_disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxp_disease_valid_dataset)

10

In [15]:
# cxr_disease_valid_dataset = CXRDataset.download_dataset(100, cxr_mode.Disease, Configs.CXR_DATA_DIR,
#                                                         Configs.CXR_VALID_LABELS_FILENAME, **Configs.CXR_FILENAMES,
#                                                         transform=valid_transform, target_transform=None)
cxr_disease_valid_dataset = CXRDataset(Mode.Disease, Configs.CXR_DATA_DIR, Configs.CXR_VALID_LABELS_FILENAME,
                                       transform=valid_transform)
cxr_disease_valid_dataset.df_labels = cxr_disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxr_disease_valid_dataloader = DataLoader(cxr_disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxr_disease_valid_dataset)

10

## Pretrained Models 

In [16]:
_, _, files = next(os.walk(Configs.CHEXPERT_DISEASE_TRAINED_MODELS_DIR))
disease_trained_models = [os.path.join(Configs.CHEXPERT_DISEASE_TRAINED_MODELS_DIR, file) for file in files]
len(disease_trained_models)

1

In [17]:
disease_model = densenet121()
num_features = disease_model.classifier.in_features
disease_model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=Configs.NUM_DISEASE_CLASSES, bias=True)
)
disease_model.eval()
not disease_model.training

True

In [18]:
disease_model, results, _, _ = shared_utils.load_statedict(disease_model, disease_trained_models[0], Configs)
disease_model = to_gpu(disease_model)

2022-07-23 19:59: Loading model - CheXpert/disease_prediction/trained_models/2022_07_12-18_47__densenet121_aug__epoch-5__iter-12659__batch_size-16__trainLastLoss-0.3754__validAUC-0.8899.dict


## Predictions

In [19]:
df_res_disease = pd.DataFrame(columns=Configs.DISEASE_ANNOTATIONS_COLUMNS + ['Mean'])
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean


In [20]:
cxp_disease_labels, cxp_disease_outputs = shared_utils.get_metric_tensors(disease_model, cxp_disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)
df_res_disease.loc['CXP'] = add_mean_to_list(shared_utils.auc_score(cxp_disease_labels, cxp_disease_outputs, per_class=True))

Only one class present in y_true. ROC AUC score is not defined in that case.


In [21]:
cxr_disease_labels, cxr_disease_outputs = shared_utils.get_metric_tensors(disease_model, cxr_disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)
df_res_disease.loc['CXR'] = add_mean_to_list(shared_utils.auc_score(cxr_disease_labels, cxr_disease_outputs, per_class=True))

Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


In [22]:
df_res_disease.sort_values(by="Mean", ascending=False, inplace=True)
df_res_disease = df_res_disease.round(2)

In [23]:
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
CXP,1.0,0.88,NaN,1.0,1.0,0.97
CXR,1.0,0.56,NaN,NaN,1.0,0.85


## Performance Per Protected Groups 

In [26]:
# race, age group, and gender to ChexPertDiseaseDatatset
cxp_demo_df = CheXpertRaceDataset.generate_race_dummies(pd.read_csv(os.path.join(Configs.CHEXPERT_DATA_DIR,
                                                                                 Configs.CHEXPERT_DEMO_FILENAME)),
                                                       'PRIMARY_RACE', Configs.CHEXPERT_RACE_DICT)
cxp_race_df = cxp_demo_df[['PATIENT', 'race'] + Configs.RACE_ANNOTATIONS_COLUMNS].drop_duplicates()
cxp_disease_df_labels = cxp_disease_valid_dataset.df_labels.merge(cxp_race_df, how='left',
                                                                                left_on='patient_id', right_on='PATIENT')
cxp_disease_df_labels.race.fillna('Other', inplace=True)
cxp_disease_df_labels['age'] = cxp_disease_df_labels.Age.apply(shared_utils.age_to_age_group)
cxp_disease_df_labels['gender'] = cxp_disease_df_labels.Sex
cxp_disease_df_labels.head(2)

,original_path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,img_path,patient_id,study,view,PATIENT,race,Asian,Black,Hispanic,White,age,gender
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64541,study1,view1_frontal.jpg,patient64541,White,0.0,0.0,0.0,1.0,70-90,Male
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64542,study1,view1_frontal.jpg,patient64542,White,0.0,0.0,0.0,1.0,70-90,Male


In [27]:
cxr_disease_df_labels = cxr_disease_valid_dataset.df_labels.copy()
cxr_disease_df_labels.gender.replace({"M": "Male", "F": "Female"}, inplace=True)
cxr_disease_df_labels.head(2)

,subject_id,study_id,split,dicom_id,ethnicity,race,age,gender,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,folder_number,img_path
0,19859532,50453930,train,a5369d17-994f6cf1-dee57320-252febcc-6906f561,ASIAN,Asian,20-40,Female,0.0,0.0,0.0,0.0,0.0,19,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...
1,17719678,54809507,train,5d2b5624-28073582-14636856-8a3597b6-4cbc4bf3,ASIAN,Asian,20-40,Female,0.0,0.0,0.0,0.0,1.0,17,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...


In [42]:
shared_utils.auc_per_protected_group(cxp_disease_df_labels, Mode.Disease, Configs, cxp_disease_labels,
                                     cxp_disease_outputs, protected_groups=['race'])

Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))
/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
race,,,,,
Asian,NaN,NaN,NaN,NaN,NaN
Other,NaN,NaN,NaN,NaN,NaN
White,1.0,0.916667,NaN,NaN,1.0


In [45]:
shared_utils.auc_per_protected_group(cxp_disease_df_labels, Mode.Disease, Configs, cxp_disease_labels,
                                     cxp_disease_outputs, protected_groups=['gender', 'age','race'])

Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.

/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))
/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))
/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


Atelectasis  Cardiomegaly  Consolidation  Edema  Pleural Effusion
gender age   race                                                                    
Female 40-70 Other          NaN           NaN            NaN    NaN               NaN
             White          NaN           NaN            NaN    NaN               NaN
Male   40-70 White          1.0      0.666667            NaN    NaN               NaN
       70-90 Asian          NaN           NaN            NaN    NaN               NaN
             White          NaN      1.000000            NaN    NaN               NaN

In [60]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['gender', 'age','race'])

Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


,,,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
gender,age,race,,,,,
Female,20-40,Asian,1.0,0.555556,NaN,NaN,1.0


# Race Prediction

## Validation Dataloaders

In [30]:
cxp_race_valid_dataset = CheXpertRaceDataset(data_dir=Configs.CHEXPERT_DATA_DIR, demo_filename=Configs.CHEXPERT_DEMO_FILENAME, 
                                             labels_filename=Configs.CHEXPERT_VALID_LABELS_FILENAME, transform=valid_transform)
cxp_race_valid_dataset.df_labels = cxp_race_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxp_race_valid_dataloader = DataLoader(cxp_race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxp_race_valid_dataset)

10

In [32]:
# cxr_disease_valid_dataset = CXRDataset.download_dataset(10, cxr_mode.Race, Configs.CXR_DATA_DIR,
#                                                         Configs.CXR_VALID_LABELS_FILENAME, **Configs.CXR_FILENAMES,
#                                                         transform=valid_transform, target_transform=None)
cxr_race_valid_dataset = CXRDataset(Mode.Race, Configs.CXR_DATA_DIR, Configs.CXR_VALID_LABELS_FILENAME,
                                    transform=valid_transform)
cxr_race_valid_dataset.df_labels = cxr_race_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxr_race_valid_dataloader = DataLoader(cxr_race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxr_race_valid_dataset)

10

## Pretrained Models 

In [33]:
_, _, files = next(os.walk(Configs.CHEXPERT_RACE_TRAINED_MODELS_DIR))
race_trained_models = [os.path.join(Configs.CHEXPERT_RACE_TRAINED_MODELS_DIR, file) for file in files]
race_model_versions = [p.split('__')[1] for p in race_trained_models]
len(race_trained_models)

8

In [34]:
models_dict = {}
for model_version, model_path in zip(race_model_versions, race_trained_models):
    model = densenet121()
    if "shallow" in model_version:
        shallow_denseblock = int(model_version.split('_')[2][10:])
        layer_offset = 3 + 2 * shallow_denseblock
        num_features = model.features[layer_offset].norm.num_features
        model = model.features[:layer_offset]
        classifier_module = nn.Sequential(
            nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),   
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=num_features, out_features=num_features, bias=True),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
        model.add_module('classifier', classifier_module)
    else:
        num_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, num_features, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
    model.eval()
    model, results, _, _ = shared_utils.load_statedict(model, model_path, Configs, device='cpu')
    models_dict[model_version] = model

2022-07-23 20:01: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-15_29__densenet121_race_denseblock1_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2805__validAUC-0.9178.dict
2022-07-23 20:01: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-02_27__densenet121_race_denseblock2_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2811__validAUC-0.9094.dict
2022-07-23 20:01: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-10_11__densenet121_race_classifier_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.732__validAUC-0.6978.dict
2022-07-23 20:01: Loading model - CheXpert/race_prediction/trained_models/2022_07_21-13_21__densenet121_race_denseblock2_shallow__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.3791__validAUC-0.9075.dict
2022-07-23 20:01: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-21_46__densenet121_race_denseblock3_freezed__epoch-9__iter-10266__batch_size-16__

## Predictions 

In [35]:
df_res_race = pd.DataFrame(columns=Configs.RACE_ANNOTATIONS_COLUMNS + ['Mean'])
df_res_race

,Asian,Black,Hispanic,White,Mean


In [36]:
for model_version, model in tqdm(models_dict.items()):
    model = to_gpu(model)
    cxp_race_labels, cxp_race_outputs = shared_utils.get_metric_tensors(model, cxp_race_valid_dataloader, Configs,
                                                                        apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                        by_study=False, challenge_ann_only=None)
    cxr_race_labels, cxr_race_outputs = shared_utils.get_metric_tensors(model, cxr_race_valid_dataloader, Configs,
                                                                        apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                        by_study=False, challenge_ann_only=None)
    df_res_race.loc[f"CXP_{model_version}"] = add_mean_to_list(shared_utils.auc_score(cxp_race_labels, cxp_race_outputs, per_class=True))
    df_res_race.loc[f"CXR_{model_version}"] = add_mean_to_list(shared_utils.auc_score(cxr_race_labels, cxr_race_outputs, per_class=True))
    model.cpu()

  0%|          | 0/8 [00:00<?, ?it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


...

/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


In [37]:
df_res_race.sort_values(by="Mean", ascending=False, inplace=True)
df_res_race = df_res_race.round(2)

In [38]:
df_res_race

,Asian,Black,Hispanic,White,Mean
CXP_densenet121_race_denseblock1_freezed,1.0,NaN,NaN,1.0,1.0
CXP_densenet121_race_denseblock2_freezed,1.0,NaN,NaN,1.0,1.0
CXP_densenet121_race_classifier_freezed,1.0,NaN,NaN,1.0,1.0
CXP_densenet121_race_denseblock2_shallow,1.0,NaN,NaN,1.0,1.0
CXP_densenet121_race_denseblock3_freezed,1.0,NaN,NaN,1.0,1.0
CXP_densenet121_race_denseblock4_freezed,1.0,NaN,NaN,1.0,1.0
CXP_densenet121_race_denseblock1_shallow,1.0,NaN,NaN,1.0,1.0
CXP_densenet121_race,1.0,NaN,NaN,1.0,1.0
CXR_densenet121_race_denseblock1_freezed,NaN,NaN,NaN,NaN,NaN
CXR_densenet121_race_denseblock2_freezed,NaN,NaN,NaN,NaN,NaN


## Performance Per Protected Groups

In [54]:
# race, age group, and gender to ChexPertDiseaseDatatset
cxp_race_df_labels = cxp_race_valid_dataset.df_labels[['PATIENT', 'race', 'AGE_AT_CXR', 'GENDER'] + Configs.RACE_ANNOTATIONS_COLUMNS].drop_duplicates()
cxp_race_df_labels['age'] = cxp_race_df_labels.AGE_AT_CXR.apply(shared_utils.age_to_age_group)
cxp_race_df_labels['gender'] = cxp_race_df_labels.GENDER
cxp_race_df_labels.head(2)

,PATIENT,race,AGE_AT_CXR,GENDER,Asian,Black,Hispanic,White,age,gender
0,patient64627,Asian,29,Female,1,0,0,0,20-40,Female
1,patient64546,White,55,Male,0,0,0,1,40-70,Male


In [56]:
cxr_race_df_labels = cxr_race_valid_dataset.df_labels.copy()
cxr_race_df_labels.gender.replace({"M": "Male", "F": "Female"}, inplace=True)
cxr_race_df_labels.head(2)

,subject_id,study_id,split,dicom_id,ethnicity,race,age,gender,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,folder_number,img_path,Asian,Black,Hispanic,White
0,19859532,50453930,train,a5369d17-994f6cf1-dee57320-252febcc-6906f561,ASIAN,Asian,20-40,Female,NaN,NaN,NaN,NaN,NaN,19,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...,1,0,0,0
1,17719678,54809507,train,5d2b5624-28073582-14636856-8a3597b6-4cbc4bf3,ASIAN,Asian,20-40,Female,NaN,NaN,NaN,NaN,1.0,17,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...,1,0,0,0


In [63]:
model = models_dict['densenet121_race']
model = to_gpu(model)
cxp_race_labels, cxp_race_outputs = shared_utils.get_metric_tensors(model, cxp_race_valid_dataloader, Configs,
                                                                    apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                    by_study=False, challenge_ann_only=None)
cxr_race_labels, cxr_race_outputs = shared_utils.get_metric_tensors(model, cxr_race_valid_dataloader, Configs,
                                                                    apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                    by_study=False, challenge_ann_only=None)

In [64]:
shared_utils.auc_per_protected_group(cxp_race_df_labels, Mode.Race, Configs, cxp_race_labels,
                                     cxp_race_outputs, protected_groups=['gender'])

Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


,Asian,Black,Hispanic,White
gender,,,,
Female,1.0,NaN,NaN,1.0
Male,NaN,NaN,NaN,NaN


In [65]:
shared_utils.auc_per_protected_group(cxp_race_df_labels, Mode.Race, Configs, cxp_race_labels,
                                     cxp_race_outputs, protected_groups=['gender', 'age'])

Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.

/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))
/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))
/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


Asian  Black  Hispanic  White
gender age                                 
Female 20-40    NaN    NaN       NaN    NaN
       40-70    NaN    NaN       NaN    NaN
       70-90    1.0    NaN       NaN    1.0
Male   40-70    NaN    NaN       NaN    NaN

In [66]:
shared_utils.auc_per_protected_group(cxr_race_df_labels, Mode.Race, Configs, cxr_race_labels,
                                     cxr_race_outputs, protected_groups=['gender', 'age'])

Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.
Only one class present in y_true. ROC AUC score is not defined in that case.


/home/student/MLH/debiasing-racial-effect-in-medical-images/shared_utils.py:161: RuntimeWarning: Mean of empty slice
  a.append(np.nanmean(a))


,,Asian,Black,Hispanic,White
gender,age,,,,
Female,20-40,NaN,NaN,NaN,NaN
